In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.set_option('display.max_columns',None)
pd.set_option('display.float_format','{:.3f}'.format)
import matplotlib.pyplot as plt
import datetime
import seaborn as sns

In [ ]:
sample_submission = pd.read_csv('../input/tabular-playground-series-may-2022/sample_submission.csv')
test = pd.read_csv('../input/tabular-playground-series-may-2022/test.csv')
train = pd.read_csv('../input/tabular-playground-series-may-2022/train.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.describe(include = 'all')

In [ ]:
test.describe(include = 'all')

In [ ]:
df = pd.concat([train,test],ignore_index = True)

In [ ]:
df_corr = df.drop(['f_27','target'],axis = 1)

In [ ]:
df_corr.describe(include='all')

In [ ]:
df_corr.info()

In [ ]:
fig = plt.figure(figsize=(20,80))

for i in range(len(df_corr.columns)-1):
    
    plt.subplot(int((len(df_corr.columns)-1)/2),2, i+1)
    plt.title(df_corr.columns[i+1])
    plt.hist(df_corr.iloc[:,i+1],bins=100)

plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(20,80))

for i in range(len(df_corr.columns)-1):
    
    plt.subplot(int((len(df_corr.columns)-1)/2),2, i+1)
    plt.title(df_corr.columns[i+1])
    sns.boxplot(df_corr.iloc[:,i+1])

plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(50,50))
sns.heatmap(df_corr.corr(),annot = True,annot_kws={'size': 15})

In [ ]:
df.corr()['target'].sort_values(ascending = False)

In [ ]:
# f_27 frequently 
df[df['f_27']=='BAAACABLBB']

In [ ]:
df_27 = df['f_27']
df_27 = pd.DataFrame(df_27)

In [ ]:
df_27

In [ ]:
df_27_loop =[[]]
for i in df_27.index:
    lis = list(df_27.loc[i,'f_27'])
    df_27_loop.append(lis)

In [ ]:
df_27_loop = pd.DataFrame(df_27_loop)
df_27_loop = df_27_loop.drop(0)

In [ ]:
for i in range(10):
    print(str(i) +str(df_27_loop.loc[:,i].unique()))

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder()
oe.fit(df_27_loop)
df_27_oe = oe.transform(df_27_loop)

In [ ]:
# from sklearn.preprocessing import OrdinalEncoder
# oe = OrdinalEncoder()
# oe.fit(df_27_loop)
# df_27_oe = oe.transform(df_27_loop)
df_27_loop = df_27_loop.reset_index()
df_27_oe = pd.get_dummies(df_27_loop)

In [ ]:
df_27 = pd.concat([df_27_oe,df_27],axis = 1)

In [ ]:
df_27['f_27_len'] = df['f_27'].apply(lambda x :len(set(x)))

In [ ]:
df = pd.concat([df_27,df],axis = 1)

In [ ]:
df.drop(['id','index','f_27'],axis = 1 ,inplace = True)

In [ ]:
x = df.drop('target',axis =1)
y = df['target']
train_x = x.loc[:899999,:]
test_x = x.loc[900000:,:]
train_y = y[:900000]

In [ ]:
# Creates a simple train split breakdown for baseline model
from sklearn.model_selection import train_test_split
test_size_pct = 0.20
X_train, X_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size = 0.2 , random_state = 42)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Dense, Input, InputLayer, Add, BatchNormalization, Dropout

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
import random

In [ ]:
%%time
# Defining model parameters...
BATCH_SIZE         = 4096
EPOCHS             = 200 
EPOCHS_COSINEDECAY = 300 
DIAGRAMS           = True
USE_PLATEAU        = False
INFERENCE          = False
VERBOSE            = 0 
TARGET             = 'target'

In [ ]:
def nn_model():
    '''
    '''
    
    activation_func = 'swish'
    inputs = Input(shape = (len(train_x.columns)))
    
    x = Dense(64, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(30e-6), 
              activation = activation_func)(inputs)
    
    #x = BatchNormalization()(x)
    
    x = Dense(64, 
          #use_bias  = True, 
          kernel_regularizer = tf.keras.regularizers.l2(30e-6), 
          activation = activation_func)(x)
    
    #x = BatchNormalization()(x)
    
    x = Dense(64, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(30e-6), 
              activation = activation_func)(x)
    
    #x = BatchNormalization()(x)
    
    x = Dense(64, 
          #use_bias  = True, 
          kernel_regularizer = tf.keras.regularizers.l2(30e-6), 
          activation = activation_func)(x)
    
    #x = BatchNormalization()(x)

    x = Dense(16, 
              #use_bias  = True, 
              kernel_regularizer = tf.keras.regularizers.l2(30e-6), 
              activation = activation_func)(x)
    
    #x = BatchNormalization()(x)

    x = Dense(1 , 
              #use_bias  = True, 
              #kernel_regularizer = tf.keras.regularizers.l2(30e-6),
              activation = 'sigmoid')(x)
    
    model = Model(inputs, x)
    
    return model

In [ ]:
def fit_model(X_train, y_train, X_val, y_val, run = 0):
    '''
    '''
    lr_start = 0.01
    start_time = datetime.datetime.now()
    
    scaler = StandardScaler()
    #scaler = RobustScaler()
    #scaler = MinMaxScaler()

    X_train = scaler.fit_transform(X_train)

    epochs = EPOCHS    
    lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.7, patience = 4, verbose = VERBOSE)
    es = EarlyStopping(monitor = 'val_loss',patience = 12, verbose = 1, mode = 'min', restore_best_weights = True)
    tm = tf.keras.callbacks.TerminateOnNaN()
    callbacks = [lr, es, tm]
    
    # Cosine Learning Rate Decay
    if USE_PLATEAU == False:
        epochs = EPOCHS_COSINEDECAY
        lr_end = 0.0002

        def cosine_decay(epoch):
            if epochs > 1:
                w = (1 + math.cos(epoch / (epochs - 1) * math.pi)) / 2
            else:
                w = 1
            return w * lr_start + (1 - w) * lr_end
        
        lr = LearningRateScheduler(cosine_decay, verbose = 0)
        callbacks = [lr, tm]
        
    model = nn_model()
    optimizer_func = tf.keras.optimizers.Adam(learning_rate = lr_start)
    loss_func = tf.keras.losses.BinaryCrossentropy()
    model.compile(optimizer = optimizer_func, loss = loss_func)
    
    X_val = scaler.transform(X_val)
    validation_data = (X_val, y_val)
    
    history = model.fit(X_train, 
                        y_train, 
                        validation_data = validation_data, 
                        epochs          = epochs,
                        verbose         = VERBOSE,
                        batch_size      = BATCH_SIZE,
                        shuffle         = True,
                        callbacks       = callbacks
                       )
    
    history_list.append(history.history)
    print(f'Training loss:{history_list[-1]["loss"][-1]:.3f}')
    callbacks, es, lr, tm, history = None, None, None, None, None
    
    
    y_val_pred = model.predict(X_val, batch_size = BATCH_SIZE, verbose = VERBOSE)
    score = roc_auc_score(y_val, y_val_pred)
    print(f'Fold {run}.{fold} | {str(datetime.datetime.now() - start_time)[-12:-7]}'
          f'| AUC: {score:.5f}')
    
    score_list.append(score)
    
    tst_data_scaled = scaler.transform(test_x)
    tst_pred = model.predict(tst_data_scaled)
    predictions.append(tst_pred)
    
    return model

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, roc_curve
import math

# Create empty lists to store NN information...
history_list = []
score_list   = []
predictions  = []

# Define kfolds for training purposes...
kf = KFold(n_splits = 5)

for fold, (trn_idx, val_idx) in enumerate(kf.split(train_x)):
    X_train, X_val = train_x.iloc[trn_idx], train_x.iloc[val_idx]
    y_train, y_val = train_y.iloc[trn_idx], train_y.iloc[val_idx]
    
    fit_model(X_train, y_train, X_val, y_val)
    
print(f'OOF AUC: {np.mean(score_list):.5f}')

In [ ]:
# Populated the prediction on the submission dataset and creates an output file
sample_submission['target'] = np.array(predictions).mean(axis = 0)
sample_submission.to_csv('my_submission.csv', index = False)